In [20]:
from dino_wm.utils.get_model import get_model
from dino_wm.planning.objectives import create_objective_fn
from dino_wm.planning.mpc import MPCPlanner
from dino_wm.planning.cem import CEMPlanner
from einops import rearrange, repeat
import einops
from dino_wm.utils.utils import move_to_device
from dino_wm.env.venv import SubprocVectorEnv
# import gymnasium as gym
# import gym_pusht
# from gym_pusht.envs import PushTEnv
import gym
import dino_wm.env
import torch
import numpy as np
from dino_wm.models.visual_world_model import VWorldModel
from dino_wm.utils.preprocessor import Preprocessor
import imageio
from torchvision import transforms





In [2]:
n_envs = 1
frameskip = 5
horizon = 5
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
wm, dataset, data_preprocessor = get_model("/home/ianchuang/dino_wm/outputs/checkpoints", "pusht", device)
wm : VWorldModel = wm
wm.to(device)
print("loaded model")

Loaded 18685 rollouts
Loaded 21 rollouts
Resuming from epoch 2: /home/ianchuang/dino_wm/outputs/checkpoints/outputs/pusht/checkpoints/model_latest.pth


Using cache found in /home/ianchuang/.cache/torch/hub/facebookresearch_dinov2_main


num_action_repeat: 1
num_proprio_repeat: 1
proprio encoder: ProprioceptiveEmbedding(
  (patch_embed): Conv1d(4, 10, kernel_size=(1,), stride=(1,))
)
action encoder: ProprioceptiveEmbedding(
  (patch_embed): Conv1d(10, 10, kernel_size=(1,), stride=(1,))
)
proprio_dim: 10, after repeat: 10
action_dim: 10, after repeat: 10
emb_dim: 404
Model emb_dim:  404
loaded model


In [ ]:
# gym.vector.SyncVectorEnv
# env = gym.vector.AsyncVectorEnv(
#     [
#         lambda: gym.make(
#             "gym_pusht/PushT-v0", 
#             disable_env_checker=True, 
#             relative=True,
#             legacy=False,
#             action_scale=100,
#             obs_type="visual_proprio", 
#             render_mode="rgb_array",
#             observation_width=224,
#             observation_height=224,
#         )
#         for _ in range(n_envs)
#     ]
# )
# obs, info = env.reset()

# env = gym.vector.AsyncVectorEnv(
#     [
#         lambda: gym.make(
#             "ian-pusht", 
#             with_velocity=True,
#             with_target=True,
#         )
#         for _ in range(n_envs)
#     ]
# )
# obs, info = env.reset()
env = gym.make(
    "ian-pusht", 
    with_velocity=True,
    with_target=True,
)

In [6]:
obs, actions, state, info = dataset[0]

obs = {
    k: v[-(frameskip*horizon):]
    for k, v in obs.items()
}
actions = actions[-(frameskip*horizon):]
state = state[-(frameskip*horizon):]
print(f"obs['visual'] shape: {obs['visual'].shape}")
print(f"obs['proprio'] shape: {obs['proprio'].shape}")
print(f"actions shape: {actions.shape}")
print(f"state shape: {state.shape}")

start_obs = {}
start_obs['visual'] = obs['visual'][:1].unsqueeze(0).repeat(n_envs, 1, 1, 1, 1)
start_obs['proprio'] = obs['proprio'][:1].unsqueeze(0).repeat(n_envs, 1, 1)
end_obs = {}
end_obs['visual'] = obs['visual'][-1:].unsqueeze(0).repeat(n_envs, 1, 1, 1, 1)
end_obs['proprio'] = obs['proprio'][-1:].unsqueeze(0).repeat(n_envs, 1, 1)
print(f"start_obs['visual'] shape: {start_obs['visual'].shape}")
print(f"start_obs['proprio'] shape: {start_obs['proprio'].shape}")
print(f"end_obs['visual'] shape: {end_obs['visual'].shape}")
print(f"end_obs['proprio'] shape: {end_obs['proprio'].shape}")

actions = einops.rearrange(actions, "(h f) a -> h (f a)", f=frameskip, h=horizon).unsqueeze(0).repeat(n_envs, 1, 1)
print(f"actions shape: {actions.shape}")


obs['visual'] shape: torch.Size([25, 3, 224, 224])
obs['proprio'] shape: torch.Size([25, 4])
actions shape: torch.Size([25, 2])
state shape: torch.Size([25, 7])
start_obs['visual'] shape: torch.Size([1, 1, 3, 224, 224])
start_obs['proprio'] shape: torch.Size([1, 1, 4])
end_obs['visual'] shape: torch.Size([1, 1, 3, 224, 224])
end_obs['proprio'] shape: torch.Size([1, 1, 4])
actions shape: torch.Size([1, 5, 10])


In [7]:
def get_inverse_normalize(mean, std):
    inv_std = [1.0/s for s in std]
    inv_mean = [-m/s for m, s in zip(mean, std)]
    return transforms.Normalize(mean=inv_mean, std=inv_std)
inverse_normalize = get_inverse_normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])

imageio.mimsave(
    "dataset.mp4",
    einops.rearrange(inverse_normalize(obs['visual']) * 255, "b c h w -> b h w c").cpu().numpy().astype(np.uint8),
    fps=10,
)

In [17]:
images = []
# env.reset(options={
#     'reset_to_state': state[0],
# })
env.reset()
env.unwrapped._set_state(state[0].tolist())
env_actions = einops.rearrange(actions, "b h (f a) -> b (h f) a", f=frameskip)
env_actions = data_preprocessor.denormalize_actions(env_actions)
for i in range(env_actions.shape[1]):
    # obs, reward, terminated, truncated, info = env.step(env_actions[:, i])
    obs, reward, done, info = env.step(env_actions[:, i].squeeze(0).tolist())
    images.append(obs['visual'])

# imageio.mimsave(
#     "env.mp4",
#     einops.rearrange(np.array(images), "n b h w c -> n h (b w) c"),
#     fps=10,
# )
imageio.mimsave(
    "env.mp4",
    images,
    fps=10,
)

In [27]:
start_obs = {
    k: v.to(device)
    for k, v in start_obs.items()
}
actions = actions.to(device)
print(f"start_obs['visual'] shape: {start_obs['visual'].shape}")
print(f"start_obs['proprio'] shape: {start_obs['proprio'].shape}")
z_obs, z = wm.rollout(start_obs, actions)

wm_obs, diff = wm.decode_obs(z_obs)
wm_images = wm_obs['visual'].squeeze(0)
wm_images = torch.clamp(wm_images, 0, 1)

imageio.mimsave(
    "imagination.mp4",
    einops.rearrange(inverse_normalize(wm_images) * 255, "b c h w -> b h w c").detach().cpu().numpy().astype(np.uint8),
    fps=10 // 5,
)

start_obs['visual'] shape: torch.Size([1, 1, 3, 224, 224])
start_obs['proprio'] shape: torch.Size([1, 1, 4])


In [ ]:
import time

def cem(wm: VWorldModel, start_obs, end_obs, actions= None):
    """
    Args:
        obs_0: (B, T, obs_dim) torch.Tensor
        obs_g: (B, T, obs_dim) torch.Tensor
        actions: (B, H, action_dim) torch.Tensor
    """
    objective_fn = create_objective_fn(alpha=1, base=1, mode="last")
    topk = 30
    var_scale = 1.0
    opt_steps = 30
    num_samples = 300
    sigma = var_scale * torch.ones(n_envs, horizon, wm.action_dim)
    mu = torch.zeros(n_envs, horizon, wm.action_dim)
    if actions is not None:
        mu[:, :actions.shape[1], :] = actions

    mu = mu.to(device)
    sigma = sigma.to(device)
    start_obs = {k: v.to(device) for k, v in start_obs.items()}
    end_obs = {k: v.to(device) for k, v in end_obs.items()}
    end_z = wm.encode_obs(end_obs)
    for env_idx in range(n_envs):
        env_start_z = {
            k: repeat(v[env_idx].unsqueeze(0), "1 ... -> n ...", n=num_samples)
            for k, v in start_obs.items()
        }
        env_end_z = {
            k: repeat(v[env_idx].unsqueeze(0), "1 ... -> n ...", n=num_samples)
            for k, v in end_z.items()
        }
        losses = []
        for _ in range(opt_steps):            
            action = torch.randn(
                num_samples, horizon, wm.action_dim, device=device
            )*sigma[env_idx] + mu[env_idx]
            action[0] = mu[env_idx]  # optional: make the first one mu itself
            with torch.no_grad():
                start_time = time.time()
                rollout_z, _ = wm.rollout(
                    obs_0=env_start_z,
                    act=action,
                )
                print(f"rollout time: {time.time() - start_time}")
                loss = objective_fn(rollout_z, env_end_z)
                losses.append(loss)
                topk_idx = torch.argsort(loss)[: topk]
                topk_action = action[topk_idx]
                mu[env_idx] = topk_action.mean(dim=0)
                sigma[env_idx] = topk_action.std(dim=0)

    return mu
    
wm_actions = cem(wm, start_obs, end_obs)

rollout time: 0.04827570915222168
rollout time: 3.7375757694244385
rollout time: 3.7386913299560547
rollout time: 3.7443952560424805
rollout time: 3.7449162006378174
rollout time: 3.7463669776916504
rollout time: 3.741705894470215
rollout time: 3.741791248321533
rollout time: 3.742806911468506
rollout time: 3.7435216903686523
rollout time: 3.744086742401123
rollout time: 3.7441606521606445
rollout time: 3.7438690662384033
rollout time: 3.746615171432495
rollout time: 3.7493979930877686
rollout time: 3.7483277320861816
rollout time: 3.7494242191314697
rollout time: 3.7479050159454346
rollout time: 3.7498514652252197
rollout time: 3.7519612312316895
rollout time: 3.7496023178100586
rollout time: 3.7497689723968506
rollout time: 3.7488958835601807
rollout time: 3.7490274906158447
rollout time: 3.7510132789611816
rollout time: 3.745695114135742
rollout time: 3.7507882118225098
rollout time: 3.751906156539917
rollout time: 3.7477924823760986
rollout time: 3.7499489784240723


In [ ]:
images = []
env.reset()
env.unwrapped._set_state(state[0].tolist())
env_actions = einops.rearrange(wm_actions, "b h (f a) -> b (h f) a", f=frameskip)
env_actions = data_preprocessor.denormalize_actions(env_actions)
for i in range(env_actions.shape[1]):
    obs, reward, done, info = env.step(env_actions[:, i].squeeze(0).tolist())
    images.append(obs['visual'])
imageio.mimsave(
    "wm_env.mp4",
    images,
    fps=10,
)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!